In [1]:
import json

data = []
with open('../data/gpt-generated-query.jsonl', 'r') as file:
    for line in file:
        data.append(json.loads(line))

data[:10]


[{'passage': "McDonald's Corporation is one of the most recognizable corporations in the world. A corporation is a company or group of people authorized to act as a single entity (legally a person) and recognized as such in law. Early incorporated entities were established by charter (i.e. by an ad hoc act granted by a monarch or passed by a parliament or legislature).",
  'old_query': '. what is a corporation?',
  'new_query': 'What is not a corporation?'},
 {'passage': 'Fertilizer burn is the result of over fertilizing plants or applying fertilizer to wet foliage. Fertilizer contains salts, which draw moisture out of plants. When you apply excess fertilizer to plants, the result is yellow or brown discoloration and root damage.',
  'old_query': 'can you burn your lawn with fertilizer',
  'new_query': 'Can you burn your lawn with fertilizer, even if you apply it properly?'},
 {'passage': "Although many lightning flashes are simply cloud-to-cloud, there are as many as 9,000,000 reporte

In [2]:
from encoder_model import TrainedEncoder

model = TrainedEncoder("BAAI/bge-small-en-v1.5")

In [5]:
negative_samples = [d['negative_queries'] for d in data]
positive_samples = [d['positive_queries'] for d in data]
passages = [d['passage'] for d in data]

negative_embeddings = model.encode(negative_samples, show_progress_bar=True)
positive_embeddings = model.encode(positive_samples, show_progress_bar=True)
passage_embeddings = model.encode(passages, show_progress_bar=True)

Inference Embeddings:   0%|          | 0/8 [00:00<?, ?it/s]


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [3]:
from tqdm import tqdm


for i in tqdm(range(len(data))):
    data[i]['passage_embedding'] = model.encode(data[i]['passage'], show_progress_bar=False)
    data[i]['negative_embedding'] = model.encode(data[i]['negative_queries'], show_progress_bar=False)
    data[i]['positive_embedding'] = model.encode(data[i]['positive_queries'], show_progress_bar=False)

100%|██████████| 8102/8102 [06:15<00:00, 21.56it/s]


In [18]:
import numpy as np

# Assuming 'arr' is the numpy 2D array we want to split into list of 1D arrays
embeddings_list = []
for i in range(len(data)):
    arr = data[i]['negative_embedding']
    data[i]['negative_embedding'] = [arr[j, :] for j in range(arr.shape[0])]

    arr = data[i]['positive_embedding']
    data[i]['positive_embedding'] = [arr[j, :] for j in range(arr.shape[0])]




AttributeError: 'list' object has no attribute 'shape'

In [30]:
for row in data:
    print(row['passage_embedding'].shape)
    for key in ['negative_embedding', 'positive_embedding']:
        print("key")
        for i in range(len(row[key])):
            print(row[key][i].shape)

(1, 384)
key
(384,)
(384,)
(384,)
(384,)
(384,)
key
(384,)
(384,)
(384,)
(384,)
(384,)
(1, 384)
key
(384,)
(384,)
(384,)
(384,)
(384,)
key
(384,)
(384,)
(384,)
(384,)
(384,)
(1, 384)
key
(384,)
(384,)
(384,)
(384,)
(384,)
key
(384,)
(384,)
(384,)
(384,)
(384,)
(1, 384)
key
(384,)
(384,)
(384,)
(384,)
(384,)
key
(384,)
(384,)
(384,)
(384,)
(384,)
(1, 384)
key
(384,)
(384,)
(384,)
(384,)
(384,)
key
(384,)
(384,)
(384,)
(384,)
(384,)
(1, 384)
key
(384,)
(384,)
(384,)
(384,)
(384,)
key
(384,)
(384,)
(384,)
(384,)
(384,)
(1, 384)
key
(384,)
(384,)
(384,)
(384,)
(384,)
key
(384,)
(384,)
(384,)
(384,)
(384,)
(1, 384)
key
(384,)
(384,)
(384,)
(384,)
(384,)
key
(384,)
(384,)
(384,)
(384,)
(384,)
(1, 384)
key
(384,)
(384,)
(384,)
(384,)
(384,)
key
(384,)
(384,)
(384,)
(384,)
(384,)
(1, 384)
key
(384,)
(384,)
(384,)
(384,)
(384,)
key
(384,)
(384,)
(384,)
(384,)
(384,)
(1, 384)
key
(384,)
(384,)
(384,)
(384,)
(384,)
key
(384,)
(384,)
(384,)
(384,)
(384,)
(1, 384)
key
(384,)
(384,)
(384,)
(384,)
(3

In [33]:
import torch

# Function to evaluate embeddings
def evaluate_embeddings(data):
    positive_similarities = []
    negative_similarities = []
    positive_similarities_rand = []
    negative_similarities_rand = []
    
    for item in tqdm(data):
        passage_emb = torch.tensor(item['passage_embedding'])[0]
        import random
        sampled_data = random.sample(data, 5)
        rand_passage_emb = [torch.tensor(sampled_item['passage_embedding'])[0] for sampled_item in sampled_data]

            # Continue with the existing code to evaluate the sampled passages
        
        # Calculate cosine similarity for positive queries
        for pos_emb in item['positive_embedding']:
            pos_emb = torch.tensor(pos_emb).reshape(-1)
            positive_similarities.append(pos_emb.dot(passage_emb) / (torch.norm(pos_emb) * torch.norm(passage_emb)))

            for rand_emb in rand_passage_emb:
                positive_similarities_rand.append(pos_emb.dot(rand_emb) / (torch.norm(pos_emb) * torch.norm(rand_emb)))

        # Calculate cosine similarity for negative queries
        for neg_emb in item['negative_embedding']:
            neg_emb = torch.tensor(neg_emb).reshape(-1)
            negative_similarities.append(neg_emb.dot(passage_emb) / (torch.norm(neg_emb) * torch.norm(passage_emb)))

            for rand_emb in rand_passage_emb:
                negative_similarities_rand.append(neg_emb.dot(rand_emb) / (torch.norm(neg_emb) * torch.norm(rand_emb)))
    
    # Calculate and print average similarities
    avg_pos_sim = torch.mean(torch.tensor(positive_similarities))
    avg_neg_sim = torch.mean(torch.tensor(negative_similarities))
    avg_pos_sim_rand = torch.mean(torch.tensor(positive_similarities_rand))
    avg_neg_sim_rand = torch.mean(torch.tensor(negative_similarities_rand))
    
    print(f"Average similarity between passages and positive queries: {avg_pos_sim:.4f}")
    print(f"Average similarity between passages and negative queries: {avg_neg_sim:.4f}")
    print(f"Average similarity between random passages and positive queries: {avg_pos_sim_rand:.4f}")
    print(f"Average similarity between random passages and negative queries: {avg_neg_sim_rand:.4f}")
    
    # Evaluate if positive queries are more similar than negative ones
    if avg_pos_sim > avg_neg_sim:
        print("Model is correctly associating positive queries closer to the passages than negative ones.")
    else:
        print("Model needs improvement. Negative queries are closer or equal to passages than positive ones.")

# Call the evaluation function
evaluate_embeddings(data)


100%|██████████| 8102/8102 [00:12<00:00, 644.21it/s]


Average similarity between passages and positive queries: 0.7160
Average similarity between passages and negative queries: 0.7294
Average similarity between random passages and positive queries: 0.4527
Average similarity between random passages and negative queries: 0.4531
Model needs improvement. Negative queries are closer or equal to passages than positive ones.
